In [14]:
#https://github.com/giswqs/geemap/blob/master/examples/notebooks/11_export_image.ipynb
import ee
import geemap
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Initialize the Earth Engine module.
ee.Initialize()

In [105]:
#// Create a domain name to attach to your output. Optional.
domain_name = 'WY'
'''
// These are the min and max corners of your domain in Lat, Long
// Western Wyoming
// Input the minimum lat, lower left corner
'''
minLat = 42.363116
#// Input the minimum long, lower left corner
minLong = -111.155208
#// Input the max lat, upper right corner
maxLat = 44.582480
#// Input the max Long, upper right corner
maxLong = -109.477849

In [106]:
#/ These are the min and max corners of your reanalysis in Lat, Long (create a slightly larger box)
#// Input the minimum lat, lower left corner
minLat2 = (minLat - 0.25);
#// print(minLat2);
#// Input the minimum long, lower left corner
minLong2 = (minLong - 0.5);
#// Input the max lat, upper right corner
maxLat2 = (maxLat + 0.25);
#// Input the max Long, upper right corner
maxLong2 = (maxLong + 0.5);

In [107]:
# This resolution for the NLCD and DEM outputs for the SnowModel domain in meters
sm_resolution = 100


'''// Resolution for the PRISM output. This shoud change by Latitude of the domain
// because the PRISM product spatial resolution is 2.5 minutes, which equals 150 arc seconds.
// You can use this arc-second calculator to estimate the correct value for the PRISM resolution by latitude
// https://opendem.info/arc2meters.html
// This is one arc-second in meters for 43 degrees N Latitude'''
one_arcsecond = 22.57
PRISM_resolution = one_arcsecond * 150

'''// Define the final output projection using EPSG codes
// These are the EPSG codes for the final projection of your SnowModel simulation.
// WGS UTM Zone 12 Code for Idaho/Wyoming = 32612
// WGS UTM Zone 11 Code for Nevada        = 32611
// WGS UTM Zone 10 Code for West Coast    = 32610
// WGS 84 4326
// WGS UTM 10
// WGS Alaska Albers = 3338'''
epsg_code = 'EPSG:32612'

#// Name the DEM output
dem_name = 'DEM'
#// Name the Land Cover output
lc_name = 'NLCD2016'

'''// The Beginning and End Dates you care about //
// This will start on the 'begin' date at 0:00 and the last iteration will be 
// on the day before the 'end' date below.'''
begin = '2014-09-01'
end = '2014-09-02'

# Domain

In [108]:
my_domain = ee.Geometry.Rectangle(**{'coords':[minLong,minLat,maxLong,maxLat],'proj': 'EPSG:4326','geodesic':True,});
my_domain2 = ee.Geometry.Rectangle([minLong2,minLat2,maxLong2,maxLat2])

In [65]:
'''////////////////   Datasets of Interest  //////////////////////
////////    Digital Elevation Models and Land Cover   /////////
///////////////////////////////////////////////////////////////

// NOTE: several choices below for DEM. Uncomment your preferred option

////////   Import 30m SRTM Data   ///////////////////
// NOTE: This only covers through 60 degrees latitude. See visualization layers.
//var SRTM30 = ee.Image('USGS/SRTMGL1_003');
// Find out some info about this image (hint: look in the console)
//var bands30 = SRTM30.bandNames();
//var info30 = SRTM30.getInfo();
//print(bands30,'Band Names');
//print(info30,'Band Info');
//Map.addLayer(SRTM30,visparams,'SRTM30');

////////  Import 100m ASTER data //////////////
// NOTE: this works above 60 deg lat; better for Alaska...
//var ASTER = ee.Image('NASA/ASTER_GED/AG100_003');
// Find out some info about this image (hint: look in the console)
//var bands100 = ASTER.bandNames();
//var info100 = ASTER.getInfo();
//print(bands100,'Band Names');
//print(info100,'Band Info');
//Map.addLayer(ASTER,visparams_aster,'ASTER');
'''
#/////////  Import 90m SRTM Data   ////////////////////
#// NOTE: This only covers through 60 degrees latitude. See visualization layers.
SRTM90 = ee.Image('CGIAR/SRTM90_V4')
bands90 = SRTM90.bandNames()
info90 = SRTM90.getInfo()
#print(bands90,'Band Names')
#print(info90,'Band Info')

In [12]:
'''////////   Import NLCD Dataset   ////////////////////
var NLCD = ee.ImageCollection('USGS/NLCD');
//Note: the NLCD has numerous images for different years.'''
landcover = NLCD.select('landcover');
#// Define the timeframe of NLCD images to select. Currently, its set to the previous 5 years.
landcoverfiltered=landcover.filterDate('2015-01-01','2020-01-01');
'''landcoverVis = {
  'min': 0.0,
  'max': 95.0,
  'palette': [
    '466b9f', 'd1def8', 'dec5c5', 'd99282', 'eb0000', 'ab0000', 'b3ac9f',
    '68ab5f', '1c5f2c', 'b5c58f', 'af963c', 'ccb879', 'dfdfc2', 'd1d182',
    'a3cc51', '82ba9e', 'dcd939', 'ab6c28', 'b8d9eb', '6c9fb8'
  ],
};'''
#// Create a single image out of the image collection using the most common land cover 
#// designation from the previous 5 years.
lcsingle=landcoverfiltered.mode();
#Map.addLayer(lcsingle, landcoverVis, 'Landcover');

# download ee image 

Here I am just downloading the dem tif, but you could also download the lc data

In [34]:
out_dir = os.path.abspath('/nfs/attic/dfh/Aragon2/Notebooks/preprocess_python/Downloads')
filename = os.path.join(out_dir, 'dem.tif')

In [39]:
image = SRTM90.clip(my_domain).unmask()
geemap.ee_export_image(image, filename=filename, scale=sm_resolution, region=my_domain, file_per_band=False, crs = epsg_code)

Generating URL ...
Please wait ...
Data downloaded to /nfs/attic/dfh/Aragon2/Notebooks/preprocess_python/Downloads/lc.tif


# download ee collection

It appears that I am able to access the data, but I have to convert the collection to images before downloading. 

In [109]:
'''////////////////   Datasets of Interest  //////////////////////
////////                Reanalysis DATA               /////////
///////////////////////////////////////////////////////////////'''
cfsv2 = ee.ImageCollection('NOAA/CFSV2/FOR6H') \
    .filterBounds(my_domain2) \
    .filter(ee.Filter.date(begin,end))


This method exports daily .tif files of the collection that have all 22 bands.
https://developers.google.com/earth-engine/datasets/catalog/NOAA_CFSV2_FOR6H#bands

In [79]:
cfsv2 = ee.ImageCollection('NOAA/CFSV2/FOR6H') \
    .filterBounds(my_domain) \
    .filter(ee.Filter.date(begin,end))
geemap.ee_export_image_collection(cfsv2,out_dir=out_dir,region=my_domain2,crs=epsg_code,scale=22200)

Total number of images: 4

Exporting 1/4: 2014090100.tif
Generating URL ...
Please wait ...
Data downloaded to /nfs/attic/dfh/Aragon2/Notebooks/preprocess_python/Downloads/2014090100.tif


Exporting 2/4: 2014090106.tif
Generating URL ...
Please wait ...
Data downloaded to /nfs/attic/dfh/Aragon2/Notebooks/preprocess_python/Downloads/2014090106.tif


Exporting 3/4: 2014090112.tif
Generating URL ...
Please wait ...
Data downloaded to /nfs/attic/dfh/Aragon2/Notebooks/preprocess_python/Downloads/2014090112.tif


Exporting 4/4: 2014090118.tif
Generating URL ...
Please wait ...
Data downloaded to /nfs/attic/dfh/Aragon2/Notebooks/preprocess_python/Downloads/2014090118.tif




This method exports daily .tif files of the selected variables needed for SM

In [111]:
cfsv2 = ee.ImageCollection('NOAA/CFSV2/FOR6H') \
    .filterBounds(my_domain2) \
    .filter(ee.Filter.date(begin,end))

data = cfsv2.select('Temperature_height_above_ground', \
    'Geopotential_height_surface', \
    'v-component_of_wind_height_above_ground', \
    'Pressure_surface', \
    'Specific_humidity_height_above_ground', \
    'Precipitation_rate_surface_6_Hour_Average', \
    'Downward_Long-Wave_Radp_Flux_surface_6_Hour_Average', \
    'Downward_Short-Wave_Radiation_Flux_surface_6_Hour_Average')
geemap.ee_export_image_collection(data, out_dir=out_dir,region=my_domain2,scale=22200,crs=epsg_code)

Total number of images: 4

Exporting 1/4: 2014090100.tif
Generating URL ...
Please wait ...
Data downloaded to /nfs/attic/dfh/Aragon2/Notebooks/preprocess_python/Downloads/2014090100.tif


Exporting 2/4: 2014090106.tif
Generating URL ...
Please wait ...
Data downloaded to /nfs/attic/dfh/Aragon2/Notebooks/preprocess_python/Downloads/2014090106.tif


Exporting 3/4: 2014090112.tif
Generating URL ...
Please wait ...
Data downloaded to /nfs/attic/dfh/Aragon2/Notebooks/preprocess_python/Downloads/2014090112.tif


Exporting 4/4: 2014090118.tif
Generating URL ...
Please wait ...
Data downloaded to /nfs/attic/dfh/Aragon2/Notebooks/preprocess_python/Downloads/2014090118.tif




# GEE data directly to numpy 
I think there should be a way to iterare through these to pull the data directly into a numpy array - but I cannot quite figure it out. 
This is a different package that could maybe be useful: https://github.com/gee-community/gee_tools/issues/27

In [96]:
tair = cfsv2.select('Temperature_height_above_ground').toBands()
elev = cfsv2.select('Geopotential_height_surface').toBands()
uwind = cfsv2.select('u-component_of_wind_height_above_ground').toBands()
vwind = cfsv2.select('v-component_of_wind_height_above_ground').toBands()
surfpres = cfsv2.select('Pressure_surface').toBands()
spechum = cfsv2.select('Specific_humidity_height_above_ground').toBands()
prec = cfsv2.select('Precipitation_rate_surface_6_Hour_Average').toBands()
lwr = cfsv2.select('Downward_Long-Wave_Radp_Flux_surface_6_Hour_Average').toBands()
swr = cfsv2.select('Downward_Short-Wave_Radiation_Flux_surface_6_Hour_Average').toBands()

In [97]:
rgb_img = geemap.ee_to_numpy(swr, region=my_domain)
rgb_img

array([[[None, None, None, None]]], dtype=object)

In [ ]:
# from geetools import batch
# import ee
# geom = ee.Geometry.Point([-72,-42]).buffer(1000)
# col = ee.ImageCollection.fromImages([ee.Image.random(1), ee.Image.random(2), ee.Image.random(3)])
# size = col.size().getInfo()
# clist = col.toList(size)
# for i in range(size):
#     image = ee.Image(clist.get(i))
#     iid = image.id().getInfo()
#     name = 'test_'+iid
#     print('downloading '+name)
#     batch.Image.toLocal(image, name, scale=100, region=geom)